# Generate the Toomre Diagram for figure

figure S7 in the original submission

In [1]:
from __future__ import print_function


import matplotlib
matplotlib.use('pdf')
    

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from astropy.io import fits
from glob import glob
from astropy.time import Time
from astropy import coordinates as coords
from astropy import units as u
from astropy import constants as const
from astropy import convolution as conv
from astropy.table import Table, Column
import scipy.interpolate as scinterp
import time
import matplotlib.patches as mp

start = time.time()


import spec_plot_tools as spt
import cal_params as cp
#import plot_spec as ps
#import bensby_plotting as bp
print(os.getcwd())

/Users/BenKaiser/Desktop/abd1714_Kaiser_et_al_2020_Science


In [2]:
plt.show()

In [3]:
#figure_output_dir='/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/First_Revision/figures'
figure_output_dir='/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/Science_versions/Second_Revision/figures'

In [4]:
#figure_output_dir='/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/ApJ_reformat/figures'

In [5]:
target_dir= '/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/'
os.chdir(target_dir)

In [6]:

npoints=100
color_dict={
    'GaiaJ1644-0449':'#ff0000',
    'SDSSJ1330+6435':'#8900ff',
    'WDJ2356-209':'#00ffc5'#'#b0ff00'
}

fill_dict={
    'GaiaJ1644-0449':'#ffc3c3',#'#ff8282',
    'SDSSJ1330+6435':'#d0a3ff',#'#c689fb',
    'WDJ2356-209':'#ebfffb'#'#d4fb7e'
}

pattern_dict={
    'GaiaJ1644-0449':'#ffc3c3',
    'SDSSJ1330+6435':'#d0a3ff',
    'WDJ2356-209':'#ebfffb'
}
step_dict={
    'GaiaJ1644-0449':10,
    'SDSSJ1330+6435':0.1,
    'WDJ2356-209':10
}
pos_dict={
    'GaiaJ1644-0449':0,
    'SDSSJ1330+6435':5,
    'WDJ2356-209':10 #'#b0ff00'
}

ssp_pos=2 #offset in addition to object offset for the case of steady-state

#making the points be in the middle of the line
for pos in pos_dict:
    pos_dict[pos]=int(npoints/2.+pos_dict[pos])



line_dict={
    'thin': ':',
    'thick':'-',
    'halo':'-.'
}

wd_marker='*'
met_marker='D'
met_color='#1ca1f2'
ci_size=14
wd_size=14
ci_leg_size=9
dp_alpha=0.5
arr_naca=[-0.4,-0.1]
alpha_range=[0.5,0.2]
arrow_segs=100
arrow_width=0.03
arrow_line=4
letter_off=-3

star_marker='o'
pop_colors=['darkorange','brown','navy','grey'] #thin disk, thick disk, halo, in-between for Bensby plots

spite_alpha=1





In [7]:
kin_file='J1644_paper_kinematics.csv'
kin_table=Table.read(kin_file)

In [8]:
#######3error distribution variables
mc_number = 100000
percent_off = 34 #1-sigma equivalent
#############
rv_zero=0.
rv_sigma=100.
#####

def get_mc_distribution(value, error):
    error_distribution = np.random.normal(loc= value, scale = error, size = mc_number)
    #plt.hist(error_distribution)
    #plt.axvline(x=value, color='r')
    #plt.axvline(x=np.median(error_distribution), linestyle='--', color='k')
    #plt.show()
    #print('value:', value, 'error:',error, 'median:', np.median(error_distribution))
    return error_distribution


def get_errors(distribution, percent_off = percent_off):
    """
    values for the error bars on the plot
    
    Returns
    [- value, + value], so to get the points on the plot where they'd be located you do
    value - [- value] , value + [- value]
    Basically these are the width of the uncertainty range on either side.
    """
    low_bar = np.nanpercentile(distribution, 50-percent_off)
    median = np.nanmedian(distribution)
    high_bar = np.nanpercentile(distribution, 50+percent_off)
    try:
        return np.array([[median-low_bar],[high_bar-median]])
    #except astropy.units.core.UnitsError as error:
    except u.core.UnitsError as error:
        return  np.array([[median.value-low_bar],[high_bar-median.value]])
    
def test_errorbars(group_vel, group_disp, label):
    group_vel=np.array(group_vel)
    group_disp=np.array(group_disp)
    u_dist= get_mc_distribution(group_vel[0], group_disp[0])
    v_dist= get_mc_distribution(group_vel[1], group_disp[1])
    w_dist=get_mc_distribution(group_vel[2], group_disp[2])
    uw_dist= np.sqrt(v_dist**2+w_dist**2)
    
    uw_mean= np.nanmean(uw_dist)
    uw_std= np.std(uw_dist)
    print(label, 'UW', uw_mean, '+/-', uw_std)
    print('quadrature version', np.sqrt(group_vel[0]**2+group_vel[2]**2),np.sqrt(group_disp[0]**2+group_disp[2]**2))
    print(label, 'V MC', np.nanmean(v_dist), '+/-', np.std(v_dist), np.median(v_dist))
    print(label, 'V reported', group_vel[1], group_disp[1])
    #plt.hist(uw_dist)
    #plt.show()
    return [group_vel[1], group_disp[1]], [uw_mean, uw_std]




In [9]:
#Torres et al. 2019 values
thin_disc_vel=[2.52, -1.45, 2.60]
thin_disc_disp=[22.5, 17.4, 16.2]
thick_disc_vel=[-18.58,-30.03, 1.06]
thick_disc_disp=[50.4, 29.3, 33.1]
halo_vel=[-27.31, -92.29, 3.41]
halo_disp=[100.8, 67.4, 66.9]
    


def generate_toomre_diagram():
    #fig= plt.figure()
    def plot_kinematic_group(group_vel, group_disp, label, color='b', marker='o'):
        group_v, group_uw= test_errorbars(group_vel, group_disp, label)
        #plt.errorbar(group_vel[1], np.sqrt(group_vel[0]**2+group_vel[2]**2), xerr= group_disp[1], yerr= np.sqrt(group_disp[0]**2+group_disp[2]**2),label=label, color=color, marker=marker)
        plt.errorbar(group_v[0], group_uw[0], xerr= group_v[1], yerr= group_uw[1] ,label=label, marker=marker,color=color, linestyle='None')
        return
    td_v, td_uw=test_errorbars(thin_disc_vel, thin_disc_disp,'thin disk')
    plot_kinematic_group(thin_disc_vel, thin_disc_disp, 'Thin Disk',color=pop_colors[0], marker=star_marker )
    plot_kinematic_group(thick_disc_vel, thick_disc_disp, 'Thick Disk', color=pop_colors[1], marker=star_marker)
    plot_kinematic_group(halo_vel, halo_disp, 'Halo',color=pop_colors[2],marker=star_marker)
    #generate 3-sigma and 5-sigma ellipse
    sig3_ellipse= mp.Ellipse((td_v[0], td_uw[0]), td_v[1]*3*2, td_uw[1]*3*2, fill=False, edgecolor='k', linestyle='--')
    sig5_ellipse= mp.Ellipse((td_v[0], td_uw[0]), td_v[1]*5*2, td_uw[1]*5*2, fill=False, edgecolor='k', linestyle='--')
    ax=plt.axes()
    ax.add_patch(sig3_ellipse)
    ax.add_patch(sig5_ellipse)
    plt.xlabel('V (km/s)')
    plt.ylabel(r'$(U^2+W^2)^{1/2}$ (km/s)')
    
    #plt.xlabel(r'$V$ $($km s$^{-1})$')
    #plt.ylabel(r'$(U^2+W^2)^{1/2}$ $($km s$^{-1})$')
    
    
    #plt.legend(loc='best')
    #plt.xlim(-300, 100)
    #plt.show()
    



In [10]:
#plt.figure(figsize=(10,10))
spt.initiate_science_plot()
#plt.figure(figsize=(7.25,7.25))
#plt.figure(figsize=(7.25,7.25),constrained_layout=True)
#plt.figure(figsize=(6.,6.),constrained_layout=True)
spt.start_ApJ_fig(width_cols=1,constrained_layout=True, width_height=[1.,1.])





generate_toomre_diagram()
for row in kin_table:
    plt.errorbar(row['v'],row['uw2'],xerr=[[row['v_err_lo']],[row['v_err_hi']]],yerr=[[row['uw2_err_lo']],[row['uw2_err_hi']]],label=row['name'],marker=wd_marker, color=color_dict[row['name']],markersize=wd_size, linestyle='None')
plt.legend(loc='upper right')
#plt.xlim(-300, 100)
plt.xlim(-200, 75)

#plt.ylim(0,200)
plt.ylim(0,225)
print(os.getcwd())
os.chdir(figure_output_dir)
print(os.getcwd())
start = time.time()
print(start)
time_string=str(start).split('.')[0]

plt.savefig('figure_toomre_'+time_string+'.pdf')#plt.grid(True)


plt.show()

thin disk UW 21.24320727845231 +/- 11.12726492135603
quadrature version 3.6208286344426743 27.725259241348855
thin disk V MC -1.4552530532148462 +/- 17.45794228021559 -1.4307007792197637
thin disk V reported -1.45 17.4
Thin Disk UW 21.175525701418305 +/- 11.131256789108356
quadrature version 3.6208286344426743 27.725259241348855
Thin Disk V MC -1.3975097417899203 +/- 17.415509035962067 -1.3918973595373627
Thin Disk V reported -1.45 17.4
Thick Disk UW 47.9263636964688 +/- 23.649620666568914
quadrature version 18.610212250267324 60.29734654194992
Thick Disk V MC -30.06317086679352 +/- 29.28578366090847 -29.99604852420974
Thick Disk V reported -30.03 29.3
Halo UW 119.86979443550123 +/- 56.43079003532772
quadrature version 27.52206750954586 120.98037030857526
Halo V MC -92.33487439182908 +/- 67.31662043416722 -92.35892250186548
Halo V reported -92.29 67.4
/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper
/Users/BenKaiser/Desktop/GaiaJ1644m0449_paper/Science_versions/Second_Revision/figures
160

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
